In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import google_api_key

In [2]:
weather_df = pd.read_csv("../WeatherPy/output_data.csv")
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Yellowknife,62.4560,-114.3525,53.56,46,52,13.00,Yellowknife,1624475484
1,Bandarbeyla,9.4942,50.8122,77.00,80,100,30.00,Bandarbeyla,1624475716
2,Tasiilaq,65.6145,-37.6368,46.56,68,98,4.29,Tasiilaq,1624475716
3,Tuktoyaktuk,69.4541,-133.0374,46.40,81,75,5.75,Tuktoyaktuk,1624475716
4,Toktogul,41.8708,72.9402,65.32,46,3,0.49,Toktogul,1624475717
...,...,...,...,...,...,...,...,...,...
553,Bayjī,34.9311,43.4870,90.54,18,0,11.77,Bayjī,1624475948
554,Narsaq,60.9167,-46.0500,48.42,88,100,8.68,Narsaq,1624475949
555,Buguruslan,53.6554,52.4420,67.89,83,100,2.21,Buguruslan,1624475949
556,Djibo,14.1022,-1.6306,95.99,29,29,14.76,Djibo,1624475949


In [3]:
## Configure gmaps
gmaps.configure(api_key=google_api_key)
locations_df = weather_df[["Lat", "Lng"]]
humidity_df = weather_df["Humidity"]

In [4]:
## Plot humidity heatmap
fig = gmaps.figure()
max_intensity = np.max(humidity_df)
fig.add_layer(gmaps.heatmap_layer(locations_df, weights = humidity_df))
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
## Create new DataFrame fitting weather criteria
narrow_df = weather_df.loc[(weather_df["Wind Speed"] <= 10) & 
                           (weather_df["Cloudiness"] == 0) & 
                           (weather_df["Max Temp"] >= 70) & 
                           (weather_df["Max Temp"] <= 80)].dropna()
narrow_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,Angoche,-16.2325,39.9086,71.87,74,0,5.50,Angoche,1624475720
63,Korla,41.7597,86.1469,74.14,24,0,5.19,Korla,1624475743
102,Marsá Maţrūḩ,31.3525,27.2453,77.11,74,0,5.06,Marsá Maţrūḩ,1624475761
110,Koslan,63.4564,48.8989,72.73,65,0,3.04,Koslan,1624475764
210,Naze,28.3667,129.4833,72.27,77,0,5.95,Naze,1624475805
304,Severnyy,50.6770,36.5532,78.84,57,0,6.71,Severnyy,1624475845
356,Kiryat Gat,31.6100,34.7642,77.81,100,0,5.79,Kiryat Gat,1624475867
358,Artyom,40.4723,50.3330,75.34,45,0,5.23,Artyom,1624475868
397,Safita,34.8205,36.1153,71.58,82,0,2.98,Safita,1624475884
411,Alexandria,31.2156,29.9553,77.61,72,0,7.40,Alexandria,1624475637


In [15]:
## Create hotel df to get hotel info
hotel_df = narrow_df.loc[:,["City","Country", "Lat", "Lng"]]

In [16]:
## Perform API call
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : google_api_key}

In [17]:
## For loop through hotel df and get city and location for api call
for index, row in hotel_df.iterrows():
    city = row["City"]
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    response = requests.get(base_url, params=params).json()
    results = response['results']
    try:
        print(f"The closest hotel in the city of {city} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing relevant info. Skipping...")
print("-----Done retrieving results-----")

The closest hotel in the city of Angoche is Hsp.
The closest hotel in the city of Korla is The Garden Hotel Korla.
The closest hotel in the city of Marsá Maţrūḩ is Negresco Hotel.
The closest hotel in the city of Koslan is Sberbank.
The closest hotel in the city of Naze is Hotel New Amami.
The closest hotel in the city of Severnyy is Hotel VOYAGE.
The closest hotel in the city of Kiryat Gat is Desert Gat.
The closest hotel in the city of Artyom is F marine resort.
The closest hotel in the city of Safita is Safita Cham Palace.
The closest hotel in the city of Alexandria is Four Seasons Hotel Alexandria at San Stefano, Egypt.
The closest hotel in the city of Tétouan is Sun Hotel.
The closest hotel in the city of Morondava is Palissandre Cote Ouest resort & SPA.
The closest hotel in the city of Dromolaxia is Sun Hall Hotel.
The closest hotel in the city of Algiers is Sofitel Algiers Hamma Garden.
Missing relevant info. Skipping...
The closest hotel in the city of Krasavino is Guest House 

In [23]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [24]:
## Plot hotel markers on top of the heatmap
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))